### Imports and File Locations

In [2]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import os
import git

In [4]:
repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)
print(os.getcwd())

c:\Users\Public\Documents\Personal_Projects\Ethan_Chalk_Project


# Exploratory Data Analysis

Read in Data

## Metadata

The original data was not formatted in the way of a long table, so I used excel to clean the data. Multiple explanatory variables were encoded in color or order of the tables, so they needed to be made explicit. I used a series of formulas for that. Thus, I created long formats of the `Chalk_Results.xlsx` and `Randomized tests.xlsx` (`Chalk_Long.xlsm` and `Metadata.slsx`) and merged and created labels for them in python. 

Variables are:
- Code: a number from 1-24. In excel, this column's color is indicative of date, and its value is also indicative of control (or chalk) and treatment. Code + bChalk form the key for merging metadata with chalk_long. 
- bChalk: a number from 1-4 (1=FL. 2=Metolius, 3=Magdust, 4=Ethan's Chalk), not directly indicative of whether chalk was used. Code + bChalk form the key for merging metadata with chalk_long.
- Pinch_Order: Order in which test was conducted, for each attempt (code number), the test was performed three times sequentially. Pinch_Order + bChalk + Code forms the primary key. 
- Pinch: The weight at which the board was pinched. 
- Pull: The weight just before the hand slipped. **Our dependent variable**. 
- Chalk: The actual chalk used, 0 is added for control. 
- Person: The person conducting the test, one intermediate and one advanced climber. 
- Day: The day when the test was conducted. Dates were, 10/30/2024,10/31/2024,11/4/2024,11/6/2024,11/8/2024,11/10/2024, and 11/14/2024. Last two were apparently "noticeably slippery". 
- Day_Order: The order in which the tests were conducted in the day. This was randomized. 


### Scratch
- Chalks 1-4: type of chalk. 
- Randomization: order (chalk/nochalk) tested
- Colors: Day indicators
- Dark red: see excel
- Run number: 1-24 Ethan, 25-48 Jackson
- Shading: 1-6 no chalk, 7-12 no chalk, number corresponds to weight in accordance w the metadata spreadsheet
- Pinches 1-3/Pulls 1-3
- Pulls 1-3: values before slip
- Pinches: watching it the whole time, pinch is x, pull is y
- supraorder maximized

In [24]:
Chalk_Long = pd.read_excel('data/Raw/Chalk_Long.xlsm', sheet_name=1, header=0)
metadata = pd.read_excel('data/Raw/Metadata.xlsx', sheet_name=1, header=0)
Chalk = pd.merge(metadata, Chalk_Long, on=["Code","bChalk"], how='left')
Chalk.to_csv('data/Processed/Chalk.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'data/Processed/Chalk.csv'

Now I need to convert some labels into factors

In [31]:
Chalk["Chalk"] = Chalk["Chalk"].astype("category")
Chalk["Chalk"] = Chalk["Chalk"].cat.rename_categories(["None", "FL", "Metolius", "Magdust", "Ethan"])
Chalk["Person"] = Chalk["Person"].astype("category")
Chalk["Pinch_Order"] = Chalk["Pinch_Order"].rank(method='dense')
Chalk["Day_Chalk"] = Chalk["Day"].astype("str") + "_" + Chalk["Chalk"].astype("str")

In [7]:
Chalk.info()
Chalk.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Code         576 non-null    int64         
 1   bChalk       576 non-null    int64         
 2   Day_Order    576 non-null    int64         
 3   Day          576 non-null    datetime64[ns]
 4   Pinch        576 non-null    float64       
 5   Pull         576 non-null    float64       
 6   Pinch_Order  576 non-null    int64         
 7   Chalk        576 non-null    category      
 8   Person       576 non-null    int64         
dtypes: category(1), datetime64[ns](1), float64(2), int64(5)
memory usage: 36.9 KB


,Code,bChalk,Day_Order,Day,Pinch,Pull,Pinch_Order,Chalk,Person
0,16,1,1,2024-10-30,4.202,34.6,1,None,1
1,16,1,1,2024-10-30,4.117,31.9,2,None,1
2,16,1,1,2024-10-30,4.154,31.9,3,None,1
3,34,1,2,2024-10-30,4.088,41.1,1,FL,2
4,34,1,2,2024-10-30,4.088,44.5,2,FL,2


# Plot Basic Graphs

Now I will plot some basic graphs and summary statistics to check data quality and statistical assumptions. 

First I will check the distributions of the numeric variables. 

In [18]:
# Graph Pinch
pinchist = px.histogram(Chalk,x="Pinch",
                        title = "Distribution of Pinch Strength")
pinchist.update_layout(
    title = {
        "text":"Distribution of Pinch Strength<br><sup>by Jerry Yu</sub>",
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    }
)
pinchist.show()
# weird shape, good coverage, no noticable outliers

In [19]:
# Graph Pull
pullhist = px.histogram(Chalk,x="Pull",
                        title = "Distribution of Pull Strength")
pullhist.update_layout(
    title = {
        "text":"Distribution of Pull Strength<br><sup>by Jerry Yu</sub>",
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    }
)
pullhist.show()
#right tail, might need normalization

In [20]:
# Now Graph Relation Between Pinch and Pull
pinchpull = px.scatter(Chalk, x = "Pinch", y = "Pull",title = "Kilograms Pulled by pinch Strength")
pinchpull.update_layout(
    title = {
        "text":"Kilograms Pulled by pinch Strength<br><sup>by Jerry Yu</sub>",
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    }
)
pinchpull.show()

#nice and Linear, but maybe not homoscedastic

In [21]:
# Now Split by Some of Our Explanatory and Nuisance Variables
byChalk = px.scatter(Chalk, x = "Pinch", y = "Pull", color = "Chalk", title = "Kilograms Pulled by pinch Strength")
byChalk.update_layout(
    title = {
        "text":"Kilograms Pulled by pinch Strength<br><sup>by Jerry Yu</sub>",
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    }
)
byChalk.show()

# Very Nice Separation, normalization probably not needed

In [ ]:
# Person
byPerson = px.scatter(Chalk, x = "Pinch", y = "Pull", color = "Person", title = "Kilograms Pulled by pinch Strength")
byPerson.update_layout(
    title = {
        "text":"Kilograms Pulled by pinch Strength<br><sup>by Jerry Yu</sub>",
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    }
)
byPerson.show()

# Clear Difference by person, as expected

In [39]:
# Test Pinch Order
byOrder = px.scatter(Chalk, x = "Pinch", y = "Pull", color = "Pinch_Order", title = "Kilograms Pulled by pinch Strength")
byOrder.update_layout(
    title = {
        "text":"Kilograms Pulled by pinch Strength<br><sup>by Jerry Yu</sub>",
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    }
)
byOrder.show()

# Very random, quite nice

We will now move on to Regression